In [ ]:
import yaml
import os
import datetime
import matplotlib.pyplot as plt
import arviz as az
import numpy as np
import pymc3 as pm
import pymc3.distributions.transforms as tr
import shutil
import theano
import theano.tensor as tt
import random
import math
import pandas as pd
from pymc3.math import *

print("Running on PyMC3 v{}".format(pm.__version__))

In [ ]:
directory = "data"

In [3]:
def get_situation(w, b):
    if 0 <= w and w <= 3:
        if 1 <= b and b <= 36:
            return 1
        elif 90 <= b and b <= 96:
            return 2
        else:
            return 3
    elif 4 <= w and w <= 6:
        if 1 <= b and b <= 36:
            return 4
        elif 90 <= b and b <= 96:
            return 5
        else:
            return 6
    else:
        if 1 <= b and b <= 36:
            return 7
        elif 90 <= b and b <= 96:
            return 8
        else:
            return 9

In [4]:
def get_outcome(r, wicket):
    if wicket is True:
        return 1
    elif r == 0:
        return 2
    elif r == 1:
        return 3
    elif r == 2:
        return 4
    elif r == 3:
        return 5
    elif r == 4:
        return 6
    elif r == 6:
        return 7
    else:
        return 0

In [5]:
data = pd.read_csv("data/deliveries.csv")

In [6]:
data.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [7]:
matches = pd.read_csv("data/matches.csv")

In [8]:
len(matches)

756

In [9]:
selected_ids = matches[matches["season"] >= 2012]["id"]

In [10]:
selected_ids

0          1
1          2
2          3
3          4
4          5
       ...  
751    11347
752    11412
753    11413
754    11414
755    11415
Name: id, Length: 298, dtype: int64

In [11]:
selected_data = data[data["match_id"].isin(selected_ids)]

In [177]:
selected_data.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [269]:
noballs_and_wides_count = 0
total_balls = 0
noballs_and_wides = {}
batsmen = []
bowlers = []
situation_data = {}

In [270]:
first_innings = selected_data[selected_data["inning"] == 1]
print(len(first_innings))

36516


In [271]:
batsman_index = {}
bowler_index = {}
batsmen = first_innings["batsman"].unique()
bowlers = first_innings["bowler"].unique()
for i in range(len(batsmen)):
    batsman_index[batsmen[i]] = i
for i in range(len(bowlers)):
    bowler_index[bowlers[i]] = i

X = [[] for i in range(9)]
id1 = [[] for i in range(9)]
id2 = [[] for i in range(9)]

for k in range(0, 8):
    noballs_and_wides[k] = 0
for l in range(0, 10):
    situation_data[l] = []

In [272]:
I = len(batsmen)
J = len(bowlers)

In [273]:
current_id = 1
w = 0
b = 0
for i in range(len(first_innings)):
    ball_data = first_innings.iloc[i]
    if current_id != ball_data["match_id"]:
        current_id = ball_data["match_id"]
        w = 0
        b = 0
        
    total_balls += 1
    b += 1
    
    batsman = ball_data["batsman"]
    bowler = ball_data["bowler"]
    
    player_dismissed = False
    if pd.notnull(first_innings["player_dismissed"].iloc[i]):
        player_dismissed = True
        w += 1
        
    if ball_data["wide_runs"] >= 1 or ball_data["noball_runs"] >= 1:
        noballs_and_wides_count += 1
        runs = ball_data["batsman_runs"] # check should be total_runs ?
        noballs_and_wides[get_outcome(runs, player_dismissed)] += 1
        continue
        
    runs = ball_data["batsman_runs"]
    l = get_situation(w, b) - 1
    k = get_outcome(runs, player_dismissed)
    if k == 0:
        continue
    X[l].append(k)
    id1[l].append(batsman_index[batsman])
    id2[l].append(bowler_index[bowler])
    situation_data[l].append((batsman_index[batsman], bowler_index[bowler], k))

In [274]:
noballs_and_wides_count

1225

In [18]:
X = np.asarray([np.array(X[i]) for i in range(9)], dtype=object)
id1 = np.asarray([np.array(id1[i]) for i in range(9)], dtype=object)
id2 = np.asarray([np.array(id2[i]) for i in range(9)], dtype=object)

In [19]:
for i in range(9):
    print(len(X[i]), len(id1[i]), len(id2[i]))

10329 10329 10329
1016 1016 1016
14562 14562 14562
47 47 47
886 886 886
7045 7045 7045
2 2 2
87 87 87
1307 1307 1307


In [20]:
print(max(X[0]))
print(min(X[0]))
print(X[0])

7
1
[2 2 6 ... 1 2 2]


In [245]:
INF = 5
testval = [[-INF + x * (2 * INF)/5.0 for x in range(6)] for i in range(0, 9)]
l = [i for i in range(9)]

In [22]:
testval

[[-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0],
 [-5.0, -3.0, -1.0, 1.0, 3.0, 5.0]]

In [23]:
model = pm.Model()
Print = tt.printing.Print("shape:")
with model:
    delta_1 = pm.Uniform("delta_1", lower=0, upper=1)
    delta_2 = pm.Uniform("delta_2", lower=0, upper=1)
    inv_sigma_sqr = pm.Gamma("sigma^-2", alpha=1.0, beta=1.0)
    inv_tau_sqr = pm.Gamma("tau^-2", alpha=1.0, beta=1.0)
    mu_1 = pm.Normal("mu_1", mu=0, sigma=1/pm.math.sqrt(inv_tau_sqr), shape=len(batsmen))
    mu_2 = pm.Normal("mu_2", mu=0, sigma=1/pm.math.sqrt(inv_tau_sqr), shape=len(bowlers))
    delta = pm.math.ge(l, 3) * delta_1 + pm.math.ge(l, 6) * delta_2
    Print(delta.shape)
    Print(mu_1.shape)
    eta = [pm.Deterministic("eta_" + str(i), mu_1[id1[i]] - mu_2[id2[i]]) for i in range(9)]
    cutpoints = pm.Normal("cutpoints", mu=0, sigma=1/pm.math.sqrt(inv_sigma_sqr), transform=pm.distributions.transforms.ordered, shape=(9,6), testval=testval)
    Print(cutpoints.shape)
    X_ = [pm.OrderedLogistic("X_" + str(i), cutpoints=cutpoints[i], eta=eta[i], observed=X[i]-1) for i in range(9)]

shape: __str__ = [9]
shape: __str__ = [232]
shape: __str__ = [9 6]


In [24]:
with model:
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [cutpoints, mu_2, mu_1, tau^-2, sigma^-2, delta_2, delta_1]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1938 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [601]:
pm.save_trace(trace)

'.pymc_4.trace'

In [597]:
with model:
    trace = pm.load_trace(".pymc_1.trace")

In [598]:
az.plot_trace(trace);

/Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/arviz/data/io_pymc3.py:89: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,
/Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/arviz/plots/traceplot.py:254: UserWarning: rcParams['plot.max_subplots'] (20) is smaller than the number of variables to plot (35770), generating only 20 plots
  UserWarning,


Error in callback <function flush_figures at 0x106571e60> (for post_execute):


KeyboardInterrupt: 

In [596]:
az.summary(trace, round_to=5)

/Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/arviz/data/io_pymc3.py:89: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu_1[0],0.24250,0.05698,0.13406,0.34712,0.00106,0.00076,2874.09789,2825.73863,2873.32268,2585.16248,1.00035
mu_1[1],0.04297,0.05950,-0.06043,0.16021,0.00113,0.00090,2763.01527,2209.64485,2748.16767,2603.32181,1.00047
mu_1[2],0.09302,0.09005,-0.07177,0.26516,0.00138,0.00125,4275.93340,2590.60527,4285.65115,2594.42750,1.00061
mu_1[3],-0.20673,0.08655,-0.35637,-0.03044,0.00149,0.00108,3373.16639,3215.97072,3372.62421,2491.20931,1.00094
mu_1[4],-0.05398,0.11179,-0.25589,0.15809,0.00176,0.00181,4027.42892,1909.83904,4064.72895,2715.20611,1.00300
...,...,...,...,...,...,...,...,...,...,...,...
"cutpoints[8,1]",-0.16610,0.06283,-0.29342,-0.05603,0.00112,0.00082,3122.28150,2964.89121,3119.69389,2787.70768,1.00160
"cutpoints[8,2]",1.25937,0.07256,1.12238,1.38991,0.00119,0.00085,3717.56692,3679.67374,3696.74844,2868.79109,0.99959
"cutpoints[8,3]",1.75898,0.08277,1.60219,1.90894,0.00129,0.00092,4108.98678,4087.99239,4114.07454,3338.43071,1.00036
"cutpoints[8,4]",1.80794,0.08415,1.64925,1.96171,0.00132,0.00093,4068.63084,4063.45247,4073.29564,3213.85780,1.00031


In [27]:
pm.save_trace(trace=trace)

'.pymc_1.trace'

In [140]:
cutpoints = np.mean(trace.get_values("cutpoints", burn=500, combine=True), axis=0)

In [161]:
cutpoints

array([[-3.28335194, -0.10171367,  1.18856103,  1.42280257,  1.45230765,
         3.29732408],
       [-3.5109235 , -0.89168927,  0.90489299,  1.34441673,  1.3682397 ,
         2.41874332],
       [-3.38672482, -0.90839266,  1.14966976,  1.63572438,  1.65024564,
         2.78703405],
       [-1.44033716,  0.48493664,  1.61143495,  2.13563816,  2.34802761,
         6.32389366],
       [-2.49014201, -0.77194113,  1.29319133,  1.73943665,  1.77555965,
         2.87891442],
       [-2.45268369, -0.67614643,  1.10615524,  1.58875475,  1.60825287,
         2.63103137],
       [-3.54358776, -2.27680704,  0.21878244,  1.65522546,  3.51078822,
         5.9665146 ],
       [-2.307594  , -0.0236675 ,  2.46999961,  3.46054005,  3.8693241 ,
         5.18679372],
       [-1.44100976, -0.16724798,  1.25845918,  1.7580189 ,  1.80687634,
         2.80411778]])

In [584]:
mu_1 = np.mean(trace.get_values("mu_1", burn=500, combine=True), axis=0)
mu_2 = np.mean(trace.get_values("mu_2", burn=500, combine=True), axis=0)

In [593]:
mu_1_sorted = sorted([(mu_1[i], batsmen[i]) for i in range(len(mu_1))])
mu_2_sorted = sorted([(mu_2[i], bowlers[i]) for i in range(len(mu_2))])

In [595]:
mu_2_sorted

[(-0.27103764734749075, 'Basil Thampi'),
 (-0.24149564027271547, 'R Vinay Kumar'),
 (-0.22764913024103603, 'JP Faulkner'),
 (-0.21488355169195072, 'P Krishna'),
 (-0.20916407073875326, 'Karanveer Singh'),
 (-0.19944083796750767, 'GS Sandhu'),
 (-0.19105533487396578, 'L Ferguson'),
 (-0.18764855639477993, 'KA Pollard'),
 (-0.18629656922605917, 'P R Barman'),
 (-0.18359313694724538, 'BB Sran'),
 (-0.1819065992331058, 'BCJ Cutting'),
 (-0.17734645803043986, 'JD Unadkat'),
 (-0.1720615288775591, 'CR Woakes'),
 (-0.16351535649011906, 'Shivam Sharma'),
 (-0.14928463721651627, 'CJ Anderson'),
 (-0.14537610428608105, 'H Viljoen'),
 (-0.14263423080570087, 'SA Abbott'),
 (-0.14122873496797717, 'MM Sharma'),
 (-0.12980988888688, 'K Gowtham'),
 (-0.12641495335678152, 'M Wood'),
 (-0.12538566594317907, 'KJ Abbott'),
 (-0.12210814107726559, 'A Singh'),
 (-0.11763851211387269, 'Parvez Rasool'),
 (-0.1107186610362957, 'JA Morkel'),
 (-0.10980872882068558, 'NLTC Perera'),
 (-0.1061030067340979, 'S Mavi

In [589]:
print(mu_1.shape)
print(mu_2.shape)

(232,)
(199,)


In [466]:
l = [i for i in range(9)]
delta_1 = np.mean(trace.get_values("delta_1", burn=500, combine=True), axis=0)
delta_2 = np.mean(trace.get_values("delta_1", burn=500, combine=True), axis=0)
delta = np.greater_equal(l, 3) * delta_1 + np.greater_equal(l, 6) * delta_2

In [467]:
delta

array([0.        , 0.        , 0.        , 0.49560971, 0.49560971,
       0.49560971, 0.99121942, 0.99121942, 0.99121942])

In [468]:
p = np.zeros(shape = (len(batsmen),len(bowlers),9,7))
for i in range(len(batsmen)):
    for j in range(len(bowlers)):
        for l in range(9):
            for k in range(7):
                if k == 0:
                    p[i,j,l,k] = 1/(1 + np.exp(-(cutpoints[l,k] - mu_1[i] + mu_2[j] - delta[l])))
                elif k == 6:
                    p[i,j,l,k] = 1 - 1/(1 + np.exp(-(cutpoints[l,k-1] - mu_1[i] + mu_2[j] - delta[l])))
                else:
                    p[i,j,l,k] = 1/(1 + np.exp(-(cutpoints[l,k] - mu_1[i] + mu_2[j] - delta[l]))) - 1/(1 + np.exp(-(cutpoints[l,k-1] - mu_1[i] + mu_2[j] - delta[l])))
                    
# can be vectorised

In [476]:
pw = np.zeros(shape=7)          
for i in range(7):
    pw[i] =  float(noballs_and_wides[i])/noballs_and_wides_count
v = float(noballs_and_wides_count)/total_balls

In [469]:
# Chosen match: Sunrisers Hyderabad vs Royal Challengers Bangalore 2017
batting_team = "Sunrisers Hyderabad"
bowling_team = "Royal Challengers Bangalore"
selected_data[(selected_data["match_id"] == 1) & (selected_data["inning"] == 1)]["batsman"].unique()

array(['DA Warner', 'S Dhawan', 'MC Henriques', 'Yuvraj Singh',
       'DJ Hooda', 'BCJ Cutting'], dtype=object)

In [470]:
for batsman in batsmen:
    if "Nehra" in batsman:
        print(batsman)

A Nehra


In [471]:
batting_order_names = ['DA Warner', 'S Dhawan', 'MC Henriques', 'Yuvraj Singh',
       'DJ Hooda', 'BCJ Cutting', 'NV Ojha', 'B Kumar', 'Bipul Sharma', 'Rashid Khan', 'A Nehra']
batting_order = []
for batsman in batting_order_names:
    batting_order.append(batsman_index[batsman])

In [472]:
batting_order

[0, 1, 2, 3, 4, 5, 55, 58, 122, 57, 142]

In [473]:
current_bowler = ""
bowling_order = []
bowling_order_names = []
for bowler in selected_data[(selected_data["match_id"] == 1) & (selected_data["inning"] == 1)]["bowler"]:
    if bowler != current_bowler:
        current_bowler = bowler
        bowling_order_names.append(bowler)
        bowling_order.append(bowler_index[bowler])

In [474]:
bowling_order_names

['TS Mills',
 'A Choudhary',
 'TS Mills',
 'YS Chahal',
 'S Aravind',
 'SR Watson',
 'YS Chahal',
 'TM Head',
 'YS Chahal',
 'A Choudhary',
 'STR Binny',
 'S Aravind',
 'A Choudhary',
 'SR Watson',
 'S Aravind',
 'YS Chahal',
 'TS Mills',
 'A Choudhary',
 'TS Mills',
 'SR Watson']

In [475]:
bowling_order

[0, 1, 0, 2, 3, 4, 2, 5, 2, 1, 6, 3, 1, 4, 3, 2, 0, 1, 0, 4]

In [482]:
selected_data.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [518]:
columns=["inning", "batting_team", "bowling_team", "over", "ball", "batsman", "non_striker", "bowler", "wide_runs", "batsman_runs", "total_runs", "player_dismissed"]

In [609]:
first_innings = []

In [610]:
X1 = [-1 for i in range(120)]
Y1 = [0 for i in range(120)]
q1 = [0 for i in range(120)]
wickets = 0
runs = 0

In [611]:
q1[0] = (batting_order[0], batting_order[1])

In [612]:
# FIRST INNINGS SIMULATION

for b in range(120):
    
    if wickets == 10:
        X1[b] = -1
        
    else:
        wide_runs = 0 
        batsman_runs = 0
        total_runs = 0
        player_dismissed = None
        
        j = bowling_order[int(b/6)]
        
        bowler = bowlers[j]
        inning = 1
        over = int(b/6)
        ball = (b%6) + 1

        while np.random.uniform(0, 1) < v:       
            random = np.random.uniform(0, 1)
            Y1[b] = (random > pw[0]) + (random > pw[0] + pw[1]) + (random > pw[0] + pw[1] + pw[2]) + \
                    (random > pw[0] + pw[1] + pw[2] + pw[3]) + (random > pw[0] + pw[1] + pw[2] + pw[3] + pw[4]) + \
                    (random > pw[0] + pw[1] + pw[2] + pw[3] + pw[4] + pw[5])
            # should fix Y1[b] replaces previous Y1[b]
            runs += 1
            
            wide_runs = 1            
            
            if Y1[b] == 0:
                wickets += 1 # fix wicket fallen on wide/noball
            elif Y1[b] == 1:
    
                runs += 0
            elif Y1[b] == 2:
                wide_runs = 1
                runs += 1
            elif Y1[b] == 3:
                wide_runs = 2
                runs += 2
            elif Y1[b] == 4:
                wide_runs = 3
                runs += 3
            elif Y1[b] == 5:
                wide_runs = 4
                runs += 4
            elif Y1[b] == 6:
                wide_runs = 5
                runs += 6  
            
            batsman = batsmen[q1[b-1][0]]
            non_striker = batsmen[q1[b-1][1]]
        
            first_innings.append([inning, batting_team, bowling_team, over, ball, batsman, non_striker, bowler, wide_runs, batsman_runs, total_runs, player_dismissed])

        #batsman to face the delivered ball
        
        if b != b0:
            if b % 6 == 0:                                              # First ball of any over
                if X1[b-1] == 1:                                        # Wicket on last ball
                    player_dismissed = batsmen[q1[b-1][0]]
                    q1[b] = (q1[b-1][1], batting_order[wickets + 1])    # non striker now on strike
                elif X1[b-1] == 2 | X1[b-1] == 4:                       # Batsmen rotated places while running btw wickets on last ball
                    q1[b] = q1[b-1]                                     # single or triple on last ball so same striker
                else:
                    q1[b] = (q1[b-1][1], q1[b-1][0])
            
            else:    
                if X1[b-1] == 1:                                        # Wicket on last ball
                    player_dismissed = batsmen[q1[b-1][0]]
                    q1[b] = (batting_order[wickets + 1], q1[b-1][1])    # new batsman on strike
                elif X1[b-1] == 3 | X1[b-1] == 5:                       # Batsmen rotated places while running btw wickets on last ball
                    q1[b] = (q1[b-1][1], q1[b-1][0])
                else:
                    q1[b] = q1[b-1]
        

        batsman = batsmen[q1[b][0]]
        non_striker = batsmen[q1[b][1]]
        first_innings.append([inning, batting_team, bowling_team, over, ball, batsman, non_striker, bowler, wide_runs, batsman_runs, total_runs, player_dismissed])
        
        #bowler to ball 
        
        l = get_situation(wickets, b) - 1
        rand = np.random.uniform(0, 1)
        
        # following can be simplified
        q = q1[b][0]
        X1[b] = 0 + (rand > p[q,j,l,0]) + (rand > (p[q,j,l,0]+p[q,j,l,1])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2]+p[q,j,l,3])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2]+p[q,j,l,3]+p[q,j,l,4])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2]+p[q,j,l,3]+p[q,j,l,4]+p[q,j,l,5]))
        print(b, X1[b])
        if X1[b] == 2:
            runs += 1
        elif X1[b] == 3:
            runs += 2
        
        elif X1[b] == 4:
            runs += 3
            
        elif X1[b] == 5 :
            runs += 4
        
        elif X1[b] == 6:
            runs += 6
            
        elif X1[b] == 0:
            wickets += 1

0 6
1 2
2 1
3 2
4 1
5 5
6 1
7 1
8 6
9 1
10 1
11 0
12 2
13 1
14 5
15 1
16 1
17 1
18 0
19 6
20 0
21 2
22 1
23 5
24 2
25 1
26 1
27 1
28 5
29 1
30 2
31 1
32 1
33 5
34 5
35 2
36 1
37 2
38 2
39 1
40 3
41 2
42 2
43 2
44 2
45 3
46 1
47 1
48 1
49 3
50 1
51 2
52 1
53 5
54 1
55 1
56 2
57 3
58 2
59 1
60 2
61 1
62 2
63 2
64 2
65 1
66 1
67 2
68 2
69 2
70 2
71 5
72 1
73 1
74 2
75 4
76 1
77 3
78 3
79 2
80 0
81 2
82 2
83 3
84 2
85 5
86 6
87 5
88 3
89 6
90 5
91 2
92 2
93 1
94 2
95 2
96 5
97 5
98 1
99 2
100 2
101 6
102 3
103 1
104 2
105 6
106 2
107 6
108 2
109 1
110 3
111 6
112 6
113 1
114 2
115 1
116 6
117 3
118 2
119 6


In [613]:
first_innings_df = pd.DataFrame(first_innings, columns = columns)

In [614]:
first_innings_df

,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,batsman_runs,total_runs,player_dismissed
0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,1,DA Warner,S Dhawan,TS Mills,0,0,0,None
1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,2,DA Warner,S Dhawan,TS Mills,1,0,0,None
2,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,2,DA Warner,S Dhawan,TS Mills,1,0,0,None
3,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,3,DA Warner,S Dhawan,TS Mills,0,0,0,None
4,1,Sunrisers Hyderabad,Royal Challengers Bangalore,0,4,S Dhawan,S Dhawan,TS Mills,0,0,0,DA Warner
...,...,...,...,...,...,...,...,...,...,...,...,...
119,1,Sunrisers Hyderabad,Royal Challengers Bangalore,19,2,BCJ Cutting,BCJ Cutting,SR Watson,0,0,0,None
120,1,Sunrisers Hyderabad,Royal Challengers Bangalore,19,3,BCJ Cutting,BCJ Cutting,SR Watson,0,0,0,BCJ Cutting
121,1,Sunrisers Hyderabad,Royal Challengers Bangalore,19,4,BCJ Cutting,BCJ Cutting,SR Watson,0,0,0,None
122,1,Sunrisers Hyderabad,Royal Challengers Bangalore,19,5,BCJ Cutting,BCJ Cutting,SR Watson,0,0,0,None


In [615]:
print("Total Runs:", runs)
print("Wickets fallen:", wickets)

Total Runs: 193
Wickets fallen: 4


In [539]:
DLS = pd.read_csv("dls.csv")

In [429]:
DLS[1:]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Wickets,Lost,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
1,1,120,100.000000,96.900000,93.000000,87.900000,81.300000,72.200000,59.900000,44.800000,29.700000,17.600000
2,2,119,99.266667,95.900000,92.116667,87.083333,80.566667,71.550000,59.333333,44.333333,29.283333,17.216667
3,3,118,98.533333,94.900000,91.233333,86.266667,79.833333,70.900000,58.766667,43.866667,28.866667,16.833333
4,4,117,97.800000,93.900000,90.350000,85.450000,79.100000,70.250000,58.200000,43.400000,28.450000,16.450000
5,5,116,97.066667,92.900000,89.466667,84.633333,78.366667,69.600000,57.633333,42.933333,28.033333,16.066667
...,...,...,...,...,...,...,...,...,...,...,...,...
116,2,5,11.416667,9.416667,8.083333,7.083333,6.083333,5.583333,5.000000,4.333333,3.500000,0.750000
117,3,4,9.133333,7.533333,6.466667,5.666667,4.866667,4.466667,4.000000,3.466667,2.800000,0.600000
118,4,3,6.850000,5.650000,4.850000,4.250000,3.650000,3.350000,3.000000,2.600000,2.100000,0.450000
119,5,2,4.566667,3.766667,3.233333,2.833333,2.433333,2.233333,2.000000,1.733333,1.400000,0.300000


In [ ]:
pd
#Duckworth-Lewis resources table (ball by ball)
library(haven)
resource.table = read.csv("C:/Users/Administrator/Downloads/twenty20.csv",header = T)
View(resource.table)
resource.table = resource.table[-1,]
resource.table = resource.table[,-c(1,2)]
resource.table = rbind(resource.table,numeric(10))
resource.table = cbind(resource.table,numeric(121))
rownames(resource.table) = 120:0
colnames(resource.table) = 10:0
r = resource.table/100
View(r)

#Resources lost due to wicket
y = array(dim = c(121,10))
for (i in 1:10){
  y[,i] = r[,i] - r[,(i+1)]
}
y = as.data.frame(y)
colnames(y) = 1:10
y = y[-121,]
View(y)

#Resources lost due to delivered ball
x = array(dim = c(120,11))
for (i in 1:120) {
  x[i,] = as.numeric(r[i,]) - as.numeric(r[(i+1),])
}
x = as.data.frame(x)
colnames(x) = 1:10
x = x[,-11]
View(x)

f = runs1
wickets2 = 0
runs2 = 0
b0 = 1
batsman_order2 = numeric(11-wickets2)             # order if battinng for remaining batsman
bowling_order2 = numeric(20- floor((b0-1)/6))     # order of bowling in remaining overs
X2 = numeric(120-b0+1)                            # vector containing output of remaining balls 
Y2 = numeric(120-b0+1)                            # vector containing output of remaining balls if wide/no ball
q2 = numeric(120-b0+1)

batsman_order2 = c(290,11,74,499,505,10,31,108,142,77,78)
bowling_order2 = c(78,366,78,366,337,385,103,337,108,337,108,337,78,385,108,366,78,385,366,385)

for(b in b0:120){
  if(wickets2 == 10){
    X2[b] = NA
  }else{
    check = 1
    while (check == 1) {
      u = runif(1)
      if(u < v){                      #wide or no ball
        
        runs2 = runs2+1
        
        random = runif(1)
        Y2[b] = 1 + ifelse(random > pw[1],1,0) + ifelse(random > (pw[1]+pw[2]),1,0) + ifelse(random > (pw[1]+pw[2]+pw[3]),1,0) + ifelse(random > (pw[1]+pw[2]+pw[3]+pw[4]),1,0) + ifelse(random > (pw[1]+pw[2]+pw[3]+pw[4]+pw[5]),1,0) + ifelse(random > (pw[1]+pw[2]+pw[3]+pw[4]+pw[5]+pw[6]),1,0)
        
        if(Y2[b] == 1){
          wickets2 = wickets2+1
        }
        if(Y2[b] == 3){
          runs2 = runs2+1
        }
        if(Y2[b] == 4){
          runs2 = runs2+2
        }
        if(Y2[b] == 5){
          runs2 = runs2+3
        }
        if(Y2[b] == 6){
          runs2 = runs2+4
        }
        if(Y2[b] == 7){
          runs2 = runs2+6
        }
        check = 1
        
      }else{
        
        #batsman to face the delivered ball
        if(b == b0){
          q2[b] = batsman_order2[1]
        }else{
          if(((b-1)/6 - as.integer((b-1)/6)) == 0){                 #First ball of an over
            if(X2[b-1] == 1){                                       #wicket on last ball
              out_batsman = q2[which(X2[1:(b-1)] == 1)]
              batsman_batted = batsman_order2[1:(wickets2+2)]
              batsman_field = setdiff(batsman_batted,out_batsman)
              q2[b] = setdiff(batsman_field,q2[b-1])
            }else{
              if(X2[b-1] == 3 | X2[b-1] == 5){                      # Batsman rotated places while running between wickets
                q2[b] = q2[b-1]
              }else{
                out_batsman = q2[which(X2[1:(b-1)] == 1)]
                batsman_batted = batsman_order2[1:(wickets2+2)]
                batsman_field = setdiff(batsman_batted,out_batsman)
                q2[b] = setdiff(batsman_field,q2[b-1])
              }
            }
          }else{
            if(X2[b-1] == 1){                                       #wicket on last ball
              q2[b] = batsman_order2[wickets2+2]
            }else{
              if(X2[b-1] == 3 | X2[b-1] == 5){                      # Batsman rotated places while running between wickets
                out_batsman = q2[which(X2[1:(b-1)] == 1)]
                batsman_batted = batsman_order2[1:(wickets2+2)]
                batsman_field = setdiff(batsman_batted,out_batsman)
                q2[b] = setdiff(batsman_field,q2[b-1])
              }else{
                q2[b] = q2[b-1]
              }
            }
          }  
        }
        
        #Bowler to ball
        j = bowling_order2[ceiling((b-b0+1)/6)]
        
        #estimation of probability of outcome in 2nd inning
        
        p2 = numeric(7)
        l = 1 + ifelse(wickets2>3,3,0) +  ifelse(wickets2>6,3,0) + ifelse(b>36,1,0) + ifelse(b>96,1,0)
        E1 = p[q2[b],j,l,3] + 2*p[q2[b],j,l,4] + 3*p[q2[b],j,l,5] + 4*p[q2[b],j,l,6] + 6*p[q2[b],j,l,7]
        E2 = x[b,(wickets2+1)] + y[b,(wickets2+1)]*p[q2[b],j,l,1]
        delta = E2/(E2 + y[b,(wickets2+1)]*(1-p[q2[b],j,l,1]-p[q2[b],j,l,2]))
        
        c = as.numeric(r[b,(wickets2+1)]*E1/((f-runs2+1)*E2))
        p2[2] = c*p[q2[b],j,l,2]
        p2[1] = p[q2[b],j,l,1] + delta*p[q2[b],j,l,2]*(1-c)
        for (k in 3:7) {
          p2[k] = ((1-p[q2[b],j,l,1]-(c + delta*(1-c))*p[q2[b],j,l,2])/(1-p[q2[b],j,l,1]-p[q2[b],j,l,2]))*p[q2[b],j,l,k]
        }
        
        
        random = runif(1)
        X2[b] = 1 + ifelse(random > p2[1],1,0) + ifelse(random > (p2[1]+p2[2]),1,0) + ifelse(random > (p2[1]+p2[2]+p2[3]),1,0) + ifelse(random > (p2[1]+p2[2]+p2[3]+p2[4]),1,0) + ifelse(random > (p2[1]+p2[2]+p2[3]+p2[4]+p2[5]),1,0) + ifelse(random > (p2[1]+p2[2]+p2[3]+p2[4]+p2[5]+p2[6]),1,0)
        
        if(X2[b] == 3){
          runs2 = runs2+1
        }
        if(X2[b] == 4){
          runs2 = runs2+2
        }
        if(X2[b] == 5){
          runs2 = runs2+3
        }
        if(X2[b] == 6){
          runs2 = runs2+4
        }
        if(X2[b] == 7){
          runs2 = runs2+6
        }
        if(X2[b] == 1){
          wickets2 = wickets2+1
        }
        check = 0
        
      }
    }
  }
}